<a href="https://colab.research.google.com/github/rpadekar2006/Simple_Python_Programs/blob/main/Assignment_2_a43.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Breadth-First Search (BFS)

BFS explores all the neighbor nodes at the present depth prior to moving on to the nodes at the next depth level. It uses a queue data structure.

In [2]:
from collections import deque

def bfs(graph, start_node):
    visited = set()
    queue = deque([start_node])
    traversal_order = []

    while queue:
        node = queue.popleft() # Dequeue a node
        if node not in visited:
            visited.add(node)
            traversal_order.append(node)
            for neighbor in graph[node]:
                if neighbor not in visited:
                    queue.append(neighbor)
    return traversal_order

start_node_bfs = 'A'
bfs_result = bfs(graph, start_node_bfs)
print(f"BFS Traversal starting from node '{start_node_bfs}': {bfs_result}")

BFS Traversal starting from node 'A': ['A', 'B', 'C', 'D', 'E', 'F']


### Depth-First Search (DFS)

DFS explores as far as possible along each branch before backtracking. It typically uses a stack data structure (often implemented recursively, but can also be iterative).

In [3]:
def dfs(graph, start_node):
    visited = set()
    stack = [start_node] # Using a list as a stack
    traversal_order = []

    while stack:
        node = stack.pop() # Pop from the end of the list
        if node not in visited:
            visited.add(node)
            traversal_order.append(node)
            # Push neighbors onto the stack. For consistent output,
            # if neighbors are sorted, process them in reverse order.
            # For this graph, we'll iterate in default order.
            for neighbor in reversed(graph[node]): # Reversed to visit in A->B->C order
                if neighbor not in visited:
                    stack.append(neighbor)
    return traversal_order

start_node_dfs = 'A'
dfs_result = dfs(graph, start_node_dfs)
print(f"DFS Traversal starting from node '{start_node_dfs}': {dfs_result}")

DFS Traversal starting from node 'A': ['A', 'B', 'D', 'E', 'F', 'C']


### A* Search Algorithm

A* is a pathfinding algorithm that uses a heuristic to guide its search. It finds the shortest path between a starting node and a goal node in a graph, considering both the cost to reach a node and the estimated cost from that node to the goal.

We'll use a slightly different graph for A* to better illustrate its pathfinding capabilities, including weighted edges.

In [4]:
graph_astar = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'D': 2, 'E': 5},
    'C': {'A': 4, 'F': 1},
    'D': {'B': 2, 'G': 1},
    'E': {'B': 5, 'G': 1},
    'F': {'C': 1, 'G': 1},
    'G': {}
}

# Heuristic function (estimated cost from node to goal)
# For A* to be optimal, the heuristic must be admissible (never overestimates the cost).
# In a real-world scenario, this would typically be pre-calculated or estimated (e.g., Euclidean distance).
heuristic = {
    'A': 7, 'B': 6, 'C': 2,
    'D': 1, 'E': 1, 'F': 1,
    'G': 0
}

print("A* Search Graph (Adjacency List with weights):")
for node, neighbors in graph_astar.items():
    print(f"{node}: {neighbors}")
print("\nHeuristic values:")
for node, h_val in heuristic.items():
    print(f"{node}: {h_val}")

A* Search Graph (Adjacency List with weights):
A: {'B': 1, 'C': 4}
B: {'A': 1, 'D': 2, 'E': 5}
C: {'A': 4, 'F': 1}
D: {'B': 2, 'G': 1}
E: {'B': 5, 'G': 1}
F: {'C': 1, 'G': 1}
G: {}

Heuristic values:
A: 7
B: 6
C: 2
D: 1
E: 1
F: 1
G: 0


In [5]:
import heapq

def a_star_search(graph, heuristic, start, goal):
    # The priority queue will store tuples of (f_cost, current_node, path)
    # f_cost = g_cost + h_cost
    # g_cost = cost from start to current_node
    # h_cost = heuristic estimate from current_node to goal
    priority_queue = [(heuristic[start], start, [start])]

    # g_cost_map stores the actual cost from start to a node
    g_cost_map = {node: float('inf') for node in graph}
    g_cost_map[start] = 0

    while priority_queue:
        f_cost, current_node, path = heapq.heappop(priority_queue)

        if current_node == goal:
            return path, g_cost_map[goal]

        for neighbor, weight in graph[current_node].items():
            # Calculate new g_cost for the neighbor
            new_g_cost = g_cost_map[current_node] + weight

            # If this path to neighbor is better than any previous one
            if new_g_cost < g_cost_map[neighbor]:
                g_cost_map[neighbor] = new_g_cost
                new_f_cost = new_g_cost + heuristic[neighbor]
                new_path = path + [neighbor]
                heapq.heappush(priority_queue, (new_f_cost, neighbor, new_path))

    return None, None # No path found


start_node_astar = 'A'
goal_node_astar = 'G'

path_astar, cost_astar = a_star_search(graph_astar, heuristic, start_node_astar, goal_node_astar)

if path_astar:
    print(f"\nPath from '{start_node_astar}' to '{goal_node_astar}': {path_astar}")
    print(f"Total cost: {cost_astar}")
else:
    print(f"\nNo path found from '{start_node_astar}' to '{goal_node_astar}'.")


Path from 'A' to 'G': ['A', 'C', 'F', 'G']
Total cost: 6


### Minimax Algorithm

The Minimax algorithm is a decision-making algorithm used in game theory, artificial intelligence, and statistics for minimizing the possible loss for a worst-case (maximum loss) scenario. It is often used in two-player, zero-sum games, such as Tic-Tac-Toe, chess, or checkers.

At each step, the algorithm considers the possible moves for each player and evaluates the outcome of those moves. It assumes that both players play optimally, with one player (Maximizer) trying to maximize their score and the other (Minimizer) trying to minimize the Maximizer's score.

In [6]:
# Representing a simple game tree
# Nodes are represented by a dictionary where:
# - 'value' is the score if it's a leaf node
# - 'children' is a list of child nodes if it's an internal node

game_tree = {
    'children': [
        {'children': [
            {'value': 3},  # Leaf node
            {'value': 5}   # Leaf node
        ]},
        {'children': [
            {'value': 2},
            {'children': [
                {'value': 9},
                {'value': 1} # Leaf node
            ]}
        ]},
        {'children': [
            {'value': 0},
            {'value': 10}
        ]}
    ]
}

print("Simple Game Tree (example structure):")
# A more readable representation might be a recursive print, but this shows the dict structure.
# We'll just print the top level for brevity.
print(game_tree)


Simple Game Tree (example structure):
{'children': [{'children': [{'value': 3}, {'value': 5}]}, {'children': [{'value': 2}, {'children': [{'value': 9}, {'value': 1}]}]}, {'children': [{'value': 0}, {'value': 10}]}]}


In [7]:
def minimax(node, depth, maximizing_player):
    # Base case: if node is a leaf or depth limit is reached
    if 'value' in node: # Check if it's a leaf node (has a value)
        return node['value']

    if depth == 0: # Assuming depth 0 means evaluating immediate children (or a shallow lookahead)
        # For simplicity, we'll assume 'value' is always present at depth 0 for this example,
        # or we'd need an evaluation function for non-leaf internal nodes.
        # Given our game_tree structure, leaf nodes have 'value'.
        # If we had a more complex game state, this would be where we call an evaluation function.
        # For this example, if depth 0 is reached on a non-leaf, we assume its children will resolve.
        pass # Let the recursion continue to children if depth is not 0

    if maximizing_player:
        max_eval = -float('inf')
        for child in node['children']:
            evaluation = minimax(child, depth - 1, False)
            max_eval = max(max_eval, evaluation)
        return max_eval
    else: # Minimizing player
        min_eval = float('inf')
        for child in node['children']:
            evaluation = minimax(child, depth - 1, True)
            min_eval = min(min_eval, evaluation)
        return min_eval

# Let's test the minimax algorithm on our game tree
# Assume the root node is a maximizing player's turn.
# The depth here indicates how many levels deep to look. Our tree is 3 levels deep.
# For example, if depth = 2, we are looking two steps ahead from the root's immediate children.
# To evaluate the entire tree as shown, we'd want a depth that covers all branches.
# Let's count the max depth of the tree.

def get_max_depth(node):
    if 'value' in node:
        return 1
    if not node['children']:
        return 1
    return 1 + max(get_max_depth(child) for child in node['children'])

max_tree_depth = get_max_depth(game_tree)

optimal_value = minimax(game_tree, max_tree_depth, True)
print(f"\nOptimal value for the maximizing player from the root of the game tree: {optimal_value}")



Optimal value for the maximizing player from the root of the game tree: 3
